## Module 3: Fourier Transform Fundamentals

In this module, we’ll explore the foundations of the Fourier transform in both continuous and discrete domains, and dive into efficient computation with the FFT.

### Part A: Continuous & Discrete FT
- **Integrals vs. sums**: Understanding how the continuous FT becomes the discrete FT  
- **Complex spectra**: Interpreting real/imaginary, magnitude/phase  
- **Aliasing & periodic extension**: Why sampling introduces wrap-around in frequency

### Part B: Discrete Fourier Transform (DFT)
- **Matrix interpretation**: Building and applying the \(N\times N\) DFT matrix  
- **Key properties**: Linearity, symmetry, convolution theorem

### Part C: Fast Fourier Transform (FFT)
- **Cooley–Tukey radix-2 algorithm**: Recursive divide-and-conquer  
- **Computational complexity**: Why FFT is \(O(N\log N)\) vs.\ DFT’s \(O(N^2)\)

---

### 📓 Notebook Demos

1. **DFT Matrix Walk-Through**  
   - Build the full \(N\times N\) DFT matrix for a toy signal (e.g.\ two sinusoids)  
   - Multiply, then visualize real/imag parts and magnitude/phase  

2. **Interactive FFT & Spectral Leakage**  
   - Vary:  
     - Number of FFT points (\(n\_fft\))  
     - Window type (rectangular, Hamming, Hann, etc.)  
   - Plot the resulting spectrum and observe how your choices affect main-lobe width and side-lobes  

3. **Zero-Padding vs.\ No Zero-Padding (Aural & Visual)**  
   - Play a short tone with and without zero-padding  
   - Overlay their spectra to see how zero-padding improves frequency resolution  

---

### 🛠 Exercise: Recursive FFT Implementation
- **Task**: Write your own radix-2 recursive FFT in Python  
- **Benchmark**: Compare its runtime against `np.fft.fft` for increasing \(N\)  
- **Visualization**: Plot runtime vs.\ \(N\) to confirm the \(O(N\log N)\) behavior


### Part A: Continuous ↔ Discrete Fourier Transform

**1. Continuous FT (integral form)**  
\[
X(f) \;=\; \int_{-\infty}^{\infty} x(t)\,e^{-j2\pi f t}\,dt
\]  
This integral decomposes a time-domain signal \(x(t)\) into continuous frequencies \(f\).

**2. Discrete FT (sum form)**  
When we sample \(x(t)\) every \(T\) seconds to get \(x[n]=x(nT)\), that integral becomes a finite sum over \(N\) points:  
\[
X[k] \;=\; \sum_{n=0}^{N-1} x[n]\,e^{-j2\pi\,k\,n/N},
\quad k=0,\dots,N-1
\]  
This is exactly the DFT—our “matrix magic” demo that follows!

**3. Complex spectrum: real/imag & magnitude/phase**  
- **Real** part → in-phase (cosine) content  
- **Imag** part → quadrature (sine) content  
- \(|X[k]|\) → how much of bin \(k\) is present  
- \(\angle X[k]\) → where in the cycle each component starts

**4. Aliasing & periodic extension**  
Because sampling “wraps” time into blocks of length \(N\), the DFT spectrum is **periodic** in \(k\). Any energy above Nyquist “folds” back and causes aliasing.

---

### Part B-1: Matrix Interpretation of the DFT

Rather than thinking of \(X[k]\) as a sum, we can see the DFT as a **linear transformation**:
1. **Build the \(N\times N\) DFT matrix**  
   \[
   W_{kn} = e^{-j2\pi\,k\,n/N}
   \]
2. **Form your signal vector**  
   \(\displaystyle x = [\,x[0],\,x[1],\dots,x[N-1]\,]^T\)
3. **Multiply**  
   \[
   X = W \,\times\, x
   \]
   Each row \(k\) of \(W\) “looks for” the contribution at frequency bin \(k\) by summing all \(x[n]\) weighted by the complex exponential.  

> **Demo 1 Preview:**  
> In the next cell we’ll actually **build** this \(W\) matrix in Python, apply it to a toy signal (two sinusoids), and then visualize:
> - real vs. imaginary parts  
> - magnitude \(\lvert X[k]\rvert\) and phase \(\angle X[k]\)  

That way you can **see** and **feel** exactly how the DFT carves your time-domain data into frequency bins.


## Demo 1: DFT Matrix Walk-Through

In this first hands-on demo we’ll build and apply the full \(N\times N\) DFT matrix to a simple toy signal (a sum of two sinusoids) and then visualize:

1. **Real & imaginary parts** of the spectrum  
2. **Magnitude** and **phase**  

**Steps**  
1. Choose a small \(N\) (e.g. 32 or 64)  
2. Construct the DFT matrix \(W_{k,n} = e^{-j2\pi kn/N}\)  
3. Define your toy signal \(x[n] = \cos(2\pi f_1 n/N) + \cos(2\pi f_2 n/N)\)  
4. Compute \(X = W\,x\)  
5. Plot:
   - \(\Re\{X\}\) and \(\Im\{X\}\)
   - \(|X|\) and \(\angle X\)

Run the cell below to see how the DFT “magic” happens under the hood.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1) Set N and frequencies
N = 64
f1, f2 = 5, 12

# 2) Build DFT matrix
n = np.arange(N)
k = n.reshape((N,1))
W = np.exp(-2j * np.pi * k * n / N)

# 3) Toy signal
x = np.cos(2*np.pi*f1*n/N) + np.cos(2*np.pi*f2*n/N)

# 4) Compute DFT
X = W @ x

# 5a) Plot real & imaginary parts
plt.figure(figsize=(10, 3))
plt.plot(n, X.real, label='Re{X}')
plt.plot(n, X.imag, label='Im{X}')
plt.title('DFT: Real & Imag Parts')
plt.xlabel('Frequency Bin k')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
plt.show()

# 5b) Plot magnitude & phase
plt.figure(figsize=(12, 3))

# Magnitude
plt.subplot(1, 2, 1)
plt.stem(n, np.abs(X), basefmt=" ")
plt.title('DFT Magnitude |X[k]|')
plt.xlabel('Frequency Bin k')
plt.ylabel('Magnitude')
plt.grid(True)

# Phase
plt.subplot(1, 2, 2)
plt.stem(n, np.angle(X), basefmt=" ")
plt.title('DFT Phase ∠X[k]')
plt.xlabel('Frequency Bin k')
plt.ylabel('Phase (radians)')
plt.grid(True)

plt.tight_layout()
plt.show()


### Interpreting the DFT Plots

**Top panel: Real vs. Imaginary Parts**  
- The blue curve shows the real part of each DFT bin, with two clear spikes at bins 5 and 12 (and their mirrored positions at \(64-5\) and \(64-12\)).  
- The orange curve is the imaginary part. It sits exactly at zero because our toy signal is the sum of two pure cosines (even, real-valued), so all imaginary components cancel out. Any tiny numerical noise is too small to see.

**Bottom left: Magnitude Spectrum \(\lvert X[k]\rvert\)**  
- You see four tall stems: two at \(k=5\) and \(12\) (the positive-frequency sinusoids) and two at \(k=59\) and \(52\) (the corresponding “negative” frequencies, i.e. \(N-5\) and \(N-12\)).  
- The height of each stem is the amplitude of that frequency component.

**Bottom right: Phase Spectrum \(\angle X[k]\)**  
- For pure cosines the phase is either \(0\) or \(\pi\) (depending on the sign of the coefficient).  
- All bins without a tone have undefined or noisy phase and are plotted accordingly.

**Why is \(\mathrm{Im}\{X\}\) exactly zero?**  
A cosine wave has no sine (quadrature) component at its exact frequencies, so when you correlate with \(e^{-j2\pi k n/N}\) the sine terms sum to zero. In other words, pure real-even signals have purely real DFT coefficients, hence the flat zero-line for \(\mathrm{Im}\{X\}\).

---

### Key Properties of the DFT

- **Linearity**  
  If \(x_1[n]\xrightarrow{\mathrm{DFT}}X_1[k]\) and \(x_2[n]\xrightarrow{\mathrm{DFT}}X_2[k]\), then for any scalars \(a,b\):  
  \[
    a\,x_1[n] + b\,x_2[n]\;\xrightarrow{\mathrm{DFT}}\;a\,X_1[k] + b\,X_2[k].
  \]  
  This means you can analyze each component separately and sum the results.

- **Symmetry (Conjugate Symmetry)**  
  For a real-valued input \(x[n]\):  
  \[
    X[N-k] = X[k]^*,
  \]  
  so the magnitude spectrum is symmetric around \(k=0\) (and \(k=N/2\) for even \(N\)), and the phase is antisymmetric.

- **Convolution Theorem**  
  Convolution in time ↔ multiplication in frequency:  
  \[
    x[n]*h[n] \;\xrightarrow{\mathrm{DFT}}\; X[k]\;\cdot\;H[k].
  \]  
  Conversely, multiplication in time ↔ circular convolution in frequency. This underpins fast filtering via the FFT.


### Part C: Fast Fourier Transform (FFT)

The FFT is an efficient algorithm for computing the DFT in \(\,O(N\log N)\) time rather than the naïve \(\,O(N^2)\) sum-of-products.

- **Cooley–Tukey radix-2 algorithm**  
  1. **Divide** the length-\(N\) signal \(x[n]\) into its even and odd indexed samples:  
     \[
       x_{\text{even}}[m]=x[2m], \quad x_{\text{odd}}[m]=x[2m+1], \quad m=0,\dots,\tfrac{N}{2}-1
     \]  
  2. **Conquer** by recursively computing two DFTs of size \(N/2\):  
     \[
       X_{\text{even}}[k]=\mathrm{DFT}_{N/2}\{x_{\text{even}}\},\quad
       X_{\text{odd}}[k]=\mathrm{DFT}_{N/2}\{x_{\text{odd}}\}.
     \]  
  3. **Combine** with “twiddle factors” \(W_N^k=e^{-j2\pi k/N}\):  
     \[
       X[k] = X_{\text{even}}[k] + W_N^k\,X_{\text{odd}}[k],\quad
       X[k+N/2] = X_{\text{even}}[k] - W_N^k\,X_{\text{odd}}[k].
     \]  

- **Recursive structure**  
  Each split halves the problem size, so you perform \(\log_2 N\) stages, each doing \(O(N)\) work.

- **Computational complexity**  
  \[
    T(N) = 2\,T\!\bigl(\tfrac{N}{2}\bigr) + O(N)
    \;\longrightarrow\; T(N)=O(N\log_2 N)
    \quad\text{vs.}\quad O(N^2)\ \text{for the direct DFT.}
  \]

---

## Interactive Spectral-Leakage Demo

Below is a single, self-contained code cell. To explore how FFT size and window shape affect spectral leakage:

1. **Modify the “USER SETTINGS”** at the very top of the cell:
   ```python
   # ── USER SETTINGS ─────────────────────────────────────────────
   n_fft       = 1024           # ← change to 64,128,256,512,1024,2048,4096, etc.
   window_type = 'hann'         # ← change to 'boxcar','hann','hamming','blackman'
   # ─────────────────────────────────────────────────────────────

In [ ]:
# ── USER SETTINGS ────────────────────────────────────────────────────────────────
n_fft       = 1024           # ← change this to 64,128,256,512,1024,2048,4096, etc.
window_type = 'hamming'         # ← change this to 'boxcar','hann','hamming','blackman'
# ────────────────────────────────────────────────────────────────────────────────

import numpy as np
import matplotlib.pyplot as plt

# Sampling config
sr       = 8000              # sampling rate (Hz)
duration =   1.0             # signal length (seconds)
t        = np.linspace(0, duration, int(sr*duration), endpoint=False)

# Toy signal: two sine‐tones at 200 Hz and 600 Hz
signal = 0.7*np.sin(2*np.pi*200*t) + 0.3*np.sin(2*np.pi*600*t)

# Build and apply window
win = np.ones(n_fft) if window_type=='boxcar' else np.hanning(n_fft)   \
      if window_type=='hann'  else np.hamming(n_fft)   \
      if window_type=='hamming' else np.blackman(n_fft)

# Zero-pad or truncate to exactly n_fft samples
sig = signal[:n_fft] if len(signal)>=n_fft else np.pad(signal, (0,n_fft-len(signal)))

# Compute FFT
S     = np.fft.rfft(sig * win, n=n_fft)
freqs = np.fft.rfftfreq(n_fft, 1/sr)
mag_db = 20*np.log10(np.abs(S)+1e-8)

# Plot
plt.figure(figsize=(10,4))
plt.plot(freqs, mag_db, alpha=0.8)
plt.title(f"Spectral Leakage — n_fft={n_fft}, window={window_type}")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude (dB)")
plt.ylim(-80, 0)
plt.grid(True)
plt.show()


## 🎵 Zero-Padding vs. No Padding (Aural & Visual)

In this demo, you will:

1. **Load** a fixed duration (e.g. 20 s) of your chosen audio clip.
2. **Zero-pad** it by an integer factor (e.g. 2×, 3×, …) to increase its length.
3. **Listen** to both the original and the padded versions side-by-side.
4. **Compute & display** their magnitude spectra in two panels for direct comparison.

---

### How to use

1. **Edit the USER SETTINGS** at the very top of the code cell:
   - `FILENAME`: the name of your audio file in `sounds/`.
   - `PADDING_FACTOR`: integer ≥ 1 (how many times longer to pad).
   - `DURATION_SEC`: number of seconds to load and play.

2. **Run the cell**.  
   - You’ll see playback widgets for the original and padded clips.
   - Below them, two subplots show their frequency-domain magnitudes.

3. **Observe** how zero-padding:
   - **Improves frequency resolution**, making spectral peaks sharper.
   - **Does not add new content**—it just interpolates more finely between bins.

Experiment with different padding factors (e.g. 1, 2, 4, 8) to see how the spectral detail changes!


In [ ]:
# ── USER SETTINGS ────────────────────────────────────────────────────────────────
FILENAME       = '22-musica-ambiente-67854.mp3'  # ← put your file in sounds/
PADDING_FACTOR = 2                               # ← integer ≥1: how many× longer to pad
DURATION_SEC   = 20                              # ← seconds to load/play
# ────────────────────────────────────────────────────────────────────────────────

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import librosa
from pathlib import Path

# ── CONFIG (don’t edit below here) ───────────────────────────────────────────────
SOUNDS_DIR = Path('sounds')

# 1) Load exactly DURATION_SEC seconds of your clip
y_full, sr   = librosa.load(str(SOUNDS_DIR / FILENAME), sr=None)
n_samples    = min(len(y_full), int(DURATION_SEC * sr))
y            = y_full[:n_samples]

# 2) Zero-pad by PADDING_FACTOR
n_orig       = len(y)
n_pad        = PADDING_FACTOR * n_orig
y_pad        = np.pad(y, (0, n_pad - n_orig), mode='constant')

# 3) Play both clips
print(f"▶️ Original clip:    {n_orig/sr:.2f}s ({n_orig} samples)")
display(Audio(data=y,     rate=sr, autoplay=False))
print(f"▶️ Zero-padded clip: {n_pad/sr:.2f}s ({n_pad} samples)")
display(Audio(data=y_pad, rate=sr, autoplay=False))

# 4) Compute their spectra
S_o   = np.fft.rfft(y,    n=n_orig)
f_o   = np.fft.rfftfreq(n_orig, 1/sr)
mag_o = 20 * np.log10(np.abs(S_o) + 1e-8)

S_p   = np.fft.rfft(y_pad, n=n_pad)
f_p   = np.fft.rfftfreq(n_pad,   1/sr)
mag_p = 20 * np.log10(np.abs(S_p) + 1e-8)

# 5) Plot side-by-side
fig, ax = plt.subplots(1, 2, figsize=(14, 4), sharey=True)

# Original
ax[0].plot(f_o, mag_o, color='C0', linewidth=1)
ax[0].set_title(f'Original (n={n_orig})')
ax[0].set_xlabel('Frequency (Hz)')
ax[0].set_ylabel('Magnitude (dB)')
ax[0].grid(True)
ax[0].set_ylim(mag_o.max() - 80, mag_o.max() + 5)

# Zero-padded
ax[1].plot(f_p, mag_p, color='C1', linewidth=1)
ax[1].set_title(f'Zero-Padded (n={n_pad})')
ax[1].set_xlabel('Frequency (Hz)')
# note: y axis label only on left subplot
ax[1].grid(True)
ax[1].set_ylim(mag_o.max() - 80, mag_o.max() + 5)

fig.suptitle('Zero-Padding vs. No Padding', fontsize=16)
plt.tight_layout(rect=[0,0,1,0.95])
plt.show()


### 🛠 Exercise: Recursive FFT Implementation

**Your challenge for Module 3:**

- **Task**  
  Implement a radix-2 recursive FFT in pure Python. Your function should:  
  1. Take a real or complex input array of length \(N\), where \(N\) is a power of two.  
  2. Split the array into even and odd-indexed samples.  
  3. Recursively compute the FFT on each half.  
  4. Combine the two halves using the twiddle factors \(e^{-j2\pi k/N}\).

- **Benchmark**  
  For a range of lengths \(N = 2^8, 2^9, \dots, 2^{15}\):  
  1. Measure the runtime of your recursive FFT.  
  2. Measure the runtime of `np.fft.fft` on the same inputs.  
  3. Record times in a table or array.

- **Visualization**  
  1. Plot both runtimes (your FFT vs. NumPy’s) as a function of \(N\) on a log–log scale.  
  2. Verify that your implementation scales like \(O(N\log N)\) while the naive DFT (if you write one) would be \(O(N^2)\).  

> **Tip:** Use Python’s `time.perf_counter()` (or `timeit`) for high-resolution timing.  
> **Bonus:** Add a third curve for a direct \(O(N^2)\) DFT to see the dramatic difference!  
